In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import re
import os
from math import isnan
from datetime import datetime
from socket import gethostname
import string
from random import *

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        return conn

In [2]:
con = create_connection(r"gilsontsl.db")

2.6.0


In [50]:
con_gilson = create_connection(r"/Volumes/npsg/tecan/SourceData/SecondStage/gilsontsl.db")

2.6.0


In [94]:
con.close()

In [10]:
TABLE_NAME = 'GILSON_SET_TABLE'

df = pd.read_sql_query(f"SELECT * from {TABLE_NAME}", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

                     RUN_DATE          ID  SAMPLE_NUMBER SAMPLE_NAME  \
0  2020-04-06 16:47:32.317022  1520030001              0    M21535_5   
1  2020-04-06 16:47:32.343616  1520030001              1    M21551_5   
2  2020-04-06 16:47:32.344137  1520030001              2    M21523_5   
3  2020-04-06 16:47:32.345212  1520030001              3    M24173_2   
4  2020-04-06 16:47:32.346140  1520030001              4    M23131_6   

  TUBE_BARCODE BROOKS_BARCODE  
0   FB02646205            127  
1   FB02646225            127  
2   FB02646185            127  
3   FB03658814            127  
4   FB02699823            127  


In [12]:
pd.read_sql_query('SELECT DISTINCT ID FROM GILSON_SET_TABLE', con)

,ID
0,1520030001
1,1520030002
2,1520030003


In [6]:
d_test = pd.read_csv('test_files/1520030001_secStg_127.tsl',sep='\t')
d_test.head()

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,0,FB02646205,NaN,YES,UNKNOWN,...,127.0,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,M21551_5,0,FB02646225,NaN,YES,UNKNOWN,...,127.0,2,P01S02


In [7]:
col_names = [re.search(r'^.+?(?=\[)', c).group(0) for c in d_test.columns]
col_names

['SkipPause',
 'MethodName',
 'SampleName',
 'SampleAmount',
 'SampleDescription',
 'SampleId',
 'InjectionYesNo',
 'PeakInformation',
 'SampleInjection',
 'NOTES_STRING',
 '#Sample Well',
 '#Plate_Sample']

In [2]:
def convert_caps_underscore(string):
    prefix_tup_int = [m.span(0) for m in re.finditer(r'^.+(?=[A-Z])', string)]
    suffix_tup_int = [m.span(0) for m in re.finditer(r'[A-Z](?:[^A-Z]*$)', string)]
    #print(prefix_tup_int, suffix_tup_int)
    p_start = prefix_tup_int[0][0]
    p_end = prefix_tup_int[0][1]
    s_start = suffix_tup_int[0][0]
    s_end = suffix_tup_int[0][1]
    return f'{string[p_start:p_end].upper()}_{string[s_start:s_end].upper()}'

In [125]:
sqlite_create_table_query = '''CREATE TABLE GILSON_TSL_TABLE (NUMBER TEXT NOT NULL,RUN_DATE TIMESTAMP,PROJECT_ID TEXT NOT NULL,PLATE_ID TEXT NOT NULL,'''

In [126]:
comma = ','
reformat_col_names = ['NUMBER', 'RUN_DATE', 'PROJECT_ID', 'PLATE_ID']
for i,c in enumerate(col_names):
    #if i == len(col_names) - 1:
    #    comma = ''
    if ord(c[0]) < 65:
        c = c[1:]
        if ' ' in c:
            prefix, suffix = c.split(' ')
            c = prefix[0].upper() + prefix[1:].lower() + suffix[0].upper() +suffix[1:].lower()
    if '_' in c:
        prefix, suffix = c.split('_')
        c = prefix[0].upper() + prefix[1:].lower() + suffix[0].upper() +suffix[1:].lower()
    new_c = convert_caps_underscore(c)
    if 'DESCRIPTION' in new_c:
        text_type = 'PRIMARY KEY'
    else:
        text_type = 'TEXT NOT NULL'
    sqlite_create_table_query += f'{new_c} {text_type}{comma}'
    reformat_col_names.append(new_c)
    
    
sqlite_create_table_query += 'GILSON_NUM TEXT NOT NULL);'
reformat_col_names.append('GILSON_NUM')

In [127]:
sqlite_create_table_query

'CREATE TABLE GILSON_TSL_TABLE (NUMBER TEXT NOT NULL,RUN_DATE TIMESTAMP,PROJECT_ID TEXT NOT NULL,PLATE_ID TEXT NOT NULL,SKIP_PAUSE TEXT NOT NULL,METHOD_NAME TEXT NOT NULL,SAMPLE_NAME TEXT NOT NULL,SAMPLE_AMOUNT TEXT NOT NULL,SAMPLE_DESCRIPTION PRIMARY KEY,SAMPLE_ID TEXT NOT NULL,INJECTIONYES_NO TEXT NOT NULL,PEAK_INFORMATION TEXT NOT NULL,SAMPLE_INJECTION TEXT NOT NULL,NOTES_STRING TEXT NOT NULL,SAMPLE_WELL TEXT NOT NULL,PLATE_SAMPLE TEXT NOT NULL,GILSON_NUM TEXT NOT NULL);'

In [128]:
reformat_col_names

['NUMBER',
 'RUN_DATE',
 'PROJECT_ID',
 'PLATE_ID',
 'SKIP_PAUSE',
 'METHOD_NAME',
 'SAMPLE_NAME',
 'SAMPLE_AMOUNT',
 'SAMPLE_DESCRIPTION',
 'SAMPLE_ID',
 'INJECTIONYES_NO',
 'PEAK_INFORMATION',
 'SAMPLE_INJECTION',
 'NOTES_STRING',
 'SAMPLE_WELL',
 'PLATE_SAMPLE',
 'GILSON_NUM']

In [129]:
con_gilson.execute(sqlite_create_table_query)

In [89]:
sqlite_delete_table_query = '''DROP TABLE GILSON_TSL_TABLE;'''

In [30]:
con.commit()
con.close()

In [123]:
con_gilson.execute(sqlite_delete_table_query)

OperationalError: no such table: GILSON_TSL_TABLE

In [ ]:
sql_delete_query = """DELETE from GILSON_TSL_TABLE""" 
con_gilson.execute(sql_delete_query)

In [194]:
pd.read_sql_query('SELECT * FROM GILSON_TSL_TABLE', con_gilson)

,NUMBER,RUN_DATE,PROJECT_ID,PLATE_ID,SKIP_PAUSE,METHOD_NAME,SAMPLE_NAME,SAMPLE_AMOUNT,SAMPLE_DESCRIPTION,SAMPLE_ID,INJECTIONYES_NO,PEAK_INFORMATION,SAMPLE_INJECTION,NOTES_STRING,SAMPLE_WELL,PLATE_SAMPLE,GILSON_NUM


In [47]:
dir_fi = [fi for fi in os.listdir('test_files/')]
dir_fi

['1520060002_secStg_15200600020.tsl',
 '1520030001_secStg_127.tsl',
 '1520030002_secStg_134.tsl',
 'gilson_set_file_127.txt',
 'gilson_set_file_134.txt',
 '1520030003_secStg_138.tsl',
 'gilson_set_file_138.txt',
 'Excel_gilson_script_test.xlsx',
 '1520060001_secStg_15200600010.tsl']

In [104]:
dtest = pd.read_csv('/Volumes/npsg/tecan/SourceData/SecondStage/15200300_01_02_127_134.tsl',sep='\t')
dtest

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,0,FB02646205,NaN,YES,UNKNOWN,...,127.0,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,M21551_5,0,FB02646225,NaN,YES,UNKNOWN,...,127.0,2,P01S02
5,RUN,4ML_10mm_2ND STAGE,M21523_5,0,FB02646185,NaN,YES,UNKNOWN,...,127.0,3,P01S03
6,RUN,4ML_10mm_2ND STAGE,M24173_2,0,FB03658814,NaN,YES,UNKNOWN,...,127.0,4,P01S04
7,RUN,4ML_10mm_2ND STAGE,M23131_6,0,FB02699823,NaN,YES,UNKNOWN,...,127.0,5,P02S01
8,RUN,4ML_10mm_2ND STAGE,M23127_6,0,FB02699828,NaN,YES,UNKNOWN,...,127.0,6,P02S02
9,RUN,4ML_10mm_2ND STAGE,L7547_3,0,FB02928213,NaN,YES,UNKNOWN,...,127.0,7,P02S03


In [103]:
dtest = pd.read_csv('/Volumes/npsg/tecan/SourceData/SecondStage/15200600_01_02_15200600010_15200600020.tsl',sep='\t')
dtest

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,L129525_6,0,FB04587339,NaN,YES,UNKNOWN,...,1.520060e+10,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,L48809_5,0,FB04251130,NaN,YES,UNKNOWN,...,1.520060e+10,2,P01S02
5,RUN,4ML_10mm_2ND STAGE,L129575_6,0,FB04716251,NaN,YES,UNKNOWN,...,1.520060e+10,3,P01S03
6,RUN,4ML_10mm_2ND STAGE,L48781_7,0,FB04250855,NaN,YES,UNKNOWN,...,1.520060e+10,4,P01S04
7,RUN,4ML_10mm_2ND STAGE,L99277_7,0,FB04242223,NaN,YES,UNKNOWN,...,1.520060e+10,5,P02S01
8,RUN,4ML_10mm_2ND STAGE,L77355_6,0,FB04363141,NaN,YES,UNKNOWN,...,1.520060e+10,6,P02S02
9,RUN,4ML_10mm_2ND STAGE,L22855_6,0,FB04575204,NaN,YES,UNKNOWN,...,1.520060e+10,7,P02S03


In [9]:
d2_test = pd.read_csv('test_files/1520060001_secStg_15200600010_edit.tsl',sep='\t')
d2_test.head()

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,L129525_6,0,FB04587339,NaN,YES,UNKNOWN,...,15200600010 / 11,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,L48809_5,0,FB04251130,NaN,YES,UNKNOWN,...,15200600010 / 11,2,P01S02


In [41]:
def grab_rows(fi_content_df):
    row_idx=[]
    for i in range(fi_content_df.shape[0]):
        try:
            if pd.isna(fi_content_df['#Plate_Sample[True,115,13]'][i]):
                pass#print('nothing')
            else:
                row_idx.append(i)
        except TypeError:
            print(i)
    return row_idx

In [77]:
def blank_barcode():
    characters = string.ascii_letters + string.punctuation  + string.digits
    blank_bc =  "BLANK_" + ''.join(choice(characters) for x in range(randint(8, 16)))
    return blank_bc 

In [119]:
def get_col_names_tbl(table_name, con):
    cursor = con.execute(f'SELECT * FROM {table_name}')
    names = list(map(lambda x: x[0], cursor.description))
    return {'length':len(names),'names':[(i,n) for i,n in enumerate(names)]}

In [48]:
def show_query(table_name, con):
    res = con.execute(f"SELECT * FROM {table_name}")
    for r in res:
        print(r)

In [137]:
def grab_colm_idx(col_name, con):
    return [i for i,n in get_col_names_tbl("GILSON_TSL_TABLE", con)['names'] if n == col_name][0]

In [207]:
def split_string_input_file(input_file):
    if '/' in input_file:
        input_file = input_file.split('/')[-1]
    return input_file

def imbue_rows(dir_fi, con, commit=False, gil_num=[gethostname()]):
    nbr = 0 
    # check to see if the file path has '/'
    tmp_dir_fi = split_string_input_file(dir_fi)
    split_fi_name = tmp_dir_fi.split('_')
    projectid = split_fi_name[0]
    # read into pandas df and coerce datatypes to string
    fi_content_df = pd.read_csv(dir_fi, sep='\t',dtype={'NOTES_STRING[False,68,9]': str, \
                                                       'SampleId[False,90,5]' : str})
    # ensure strings for each value
    fi_content_df['SampleId[False,90,5]'] = fi_content_df['SampleId[False,90,5]'].apply(lambda x: 'NaN')
    fi_content_df['SampleAmount[False,78,3]'] = fi_content_df['SampleAmount[False,78,3]'].apply(lambda x: f"{str(x)}")
    fi_content_df['#Sample Well[True,109,10]'] = fi_content_df['#Sample Well[True,109,10]'].apply(lambda x: f"{str(x)}")
    # split string and create new column
    fi_content_df['plateid'] = fi_content_df['NOTES_STRING[False,68,9]'].str.split('/').str[1]
    # drop the original column since we got the projecti from the file already
    fi_content_df.drop('NOTES_STRING[False,68,9]', axis=1, inplace=True)
    current_col_names = fi_content_df.columns.tolist()[:-1]
    # remove plateid colm as separate list
    plateid_list = [x for x in fi_content_df.plateid.tolist() if not pd.isna(x)]
    # slice df to exlucde plateid
    fi_content_df = fi_content_df[current_col_names]
    
    # get the valid row indices (non-flush/std/shutdown)
    row_idx = grab_rows(fi_content_df)
    len_rows = len(row_idx)
    # get length of gilson table to import to
    len_cols = get_col_names_tbl('GILSON_TSL_TABLE', con)['length']
    # if not a full set, add blanks 
    if len_rows % 4 != 0:
        len_rows = len_rows + (4-len_rows%4)
    # the index nbr for the blank sample well
    blank_sample_idx = grab_colm_idx('SAMPLE_WELL', con) 
    # add random blank barcode
    blank_bc_idx = grab_colm_idx('SAMPLE_DESCRIPTION', con) 
    
    for i in range(len_rows): 
        nbr += 1
        row_data = ["BLANK"]*len_cols
        row_data[blank_sample_idx] = nbr
        row_data[2] = projectid
        row_data[0] = nbr
        row_data[-1] = gil_num[0]
        row_data[blank_bc_idx] = blank_barcode()
        try:
            row_data =  list(map(lambda x: str(x), fi_content_df.iloc[row_idx[i]].values.tolist())) + gil_num 
            first_4_vals = [str(nbr), datetime.now().strftime("%Y-%b-%d %H:%M:%S"), projectid, plateid_list[i]]
            row_data = first_4_vals + row_data
        except IndexError as e:
            pass
        if not commit:
            print(row_data)

        #print(all([type(x) == str for x in row_data]))
        #print(tuple(row_data))
        if commit:
            try:
                con.execute("INSERT INTO GILSON_TSL_TABLE VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row_data)
            except:
                print(row_data)
    if commit:
        con.commit()

    #return df_list

In [43]:
def grab_rows(fi_content_df):
    '''Grab the valid rows from the worklist file'''
    row_idx=[]
    for i in range(fi_content_df.shape[0]):
        try:
            if pd.isna(fi_content_df['#Plate_Sample[True,115,13]'][i]):
                pass#print('nothing')
            
            else:
                row_idx.append(i)
        except TypeError:
            print(i)
    return row_idx
    
def blank_barcode():
    '''Create blank barcodes to satisfy the database integrity'''
    blank_bc =  "BLANK_" + ''.join(choice(characters) for x in range(randint(8, 16)))
    return blank_bc 


def get_col_names_tbl(table_name, con):
    '''get length of column names and content with indices; returns dictionary'''
    cursor = con.execute(f'SELECT * FROM {table_name}')
    names = list(map(lambda x: x[0], cursor.description))
    return {'length':len(names),'names':[(i,n) for i,n in enumerate(names)]}
    
    
def grab_colm_idx(col_name, con):
    '''returns just the column index that matches the query name'''
    return [i for i,n in get_col_names_tbl("GILSON_TSL_TABLE", con)['names'] if n == col_name][0]
    
def split_string_input_file(input_file):
    '''handles if contains filepath separator'''
    if '/' in input_file:
        input_file = input_file.split('/')[-1]
    return input_file


def imbue_rows(dir_fi, con, commit=False, gil_num=[gethostname()]):
    '''print out each row after extracting data from raw worklist file and then repare for sql entry'''
    nbr = 0 
    # check to see if the file path has '/'
    tmp_dir_fi = split_string_input_file(dir_fi)
    split_fi_name = tmp_dir_fi.split('_')
    projectid = split_fi_name[0]
    # read into pandas df and coerce datatypes to string
    fi_content_df = pd.read_csv(dir_fi, sep='\t',dtype={'NOTES_STRING[False,68,9]': str, \
                                                       'SampleId[False,90,5]' : str})
    # ensure strings for each value
    fi_content_df['SampleId[False,90,5]'] = fi_content_df['SampleId[False,90,5]'].apply(lambda x: 'NaN')
    fi_content_df['SampleAmount[False,78,3]'] = fi_content_df['SampleAmount[False,78,3]'].apply(lambda x: f"{str(x)}")
    fi_content_df['#Sample Well[True,109,10]'] = fi_content_df['#Sample Well[True,109,10]'].apply(lambda x: f"{str(x)}")
    # split string and create new column
    fi_content_df['plateid'] = fi_content_df['NOTES_STRING[False,68,9]'].str.split('/').str[1]
    # drop the original column since we got the projecti from the file already
    fi_content_df.drop('NOTES_STRING[False,68,9]', axis=1, inplace=True)
    current_col_names = fi_content_df.columns.tolist()[:-1]
    # remove plateid colm as separate list
    plateid_list = [x for x in fi_content_df.plateid.tolist() if not pd.isna(x)]
    # slice df to exlucde plateid
    fi_content_df = fi_content_df[current_col_names]
    
    # get the valid row indices (non-flush/std/shutdown)
    row_idx = grab_rows(fi_content_df)
    len_rows = len(row_idx)
    # get length of gilson table to import to
    len_cols = get_col_names_tbl('GILSON_TSL_TABLE', con)['length']
    # if not a full set, add blanks 
    if len_rows % 4 != 0:
        len_rows = len_rows + (4-len_rows%4)
    # the index nbr for the blank sample well
    blank_sample_idx = grab_colm_idx('SAMPLE_WELL', con) 
    # add random blank barcode
    blank_bc_idx = grab_colm_idx('SAMPLE_DESCRIPTION', con) 
    
    for i in range(len_rows): 
        nbr += 1
        row_data = ["BLANK"]*len_cols
        row_data[blank_sample_idx] = nbr
        row_data[2] = projectid
        row_data[0] = nbr
        row_data[-1] = gil_num[0]
        row_data[blank_bc_idx] = blank_barcode()
        print(row_data)
        try:
            row_data =  list(map(lambda x: str(x), fi_content_df.iloc[row_idx[i]].values.tolist())) + gil_num 
            first_4_vals = [str(nbr), datetime.now().strftime("%Y-%b-%d %H:%M:%S"), projectid, plateid_list[i]]
            row_data = first_4_vals + row_data
        except IndexError as e:
             print('passing...')
        print(row_data)

In [45]:

characters = string.ascii_letters + string.punctuation  + string.digits

In [46]:
imbue_rows(f'test_files/1520060001_secStg_15200600010_edit.tsl', con)

[1, 'BLANK', '1520060001', 'BLANK', 'BLANK', 'BLANK', 'BLANK', "BLANK_IKA'P_T>!REY7iUl", 'BLANK', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 1, 'BLANK', 'FR-N-S081093']
['1', '2020-Jun-26 17:52:21', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L129525_6', '0', 'FB04587339', 'NaN', 'YES', 'UNKNOWN', '...', '1', 'P01S01', 'FR-N-S081093']
[2, 'BLANK', '1520060001', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 'BLANK_[#(xm1l"pu*nAM%-', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 2, 'BLANK', 'FR-N-S081093']
['2', '2020-Jun-26 17:52:21', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L48809_5', '0', 'FB04251130', 'NaN', 'YES', 'UNKNOWN', '...', '2', 'P01S02', 'FR-N-S081093']
[3, 'BLANK', '1520060001', 'BLANK', 'BLANK', 'BLANK', 'BLANK', "BLANK_p+mi=/QjV'\\/nIC", 'BLANK', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 3, 'BLANK', 'FR-N-S081093']
['3', '2020-Jun-26 17:52:21', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L129575_6', '0', 'FB04716251', 'NaN', 'YES', 'UNKNOWN', '...', '3', 'P01S03', 'FR-N-

In [190]:
imbue_rows(f'test_files/{dir_fi[-1]}', con, True)

ProgrammingError: Cannot operate on a closed database.

In [51]:
show_query('GILSON_TSL_TABLE', con_gilson)

('1', '2020-Jun-26 18:00:25', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L129525_6', '0', 'FB04587339', 'NaN', 'YES', 'UNKNOWN', '...', '15200600010 / 11', '1', 'P01S01', 'FR-N-S081093')
('2', '2020-Jun-26 18:00:25', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L48809_5', '0', 'FB04251130', 'NaN', 'YES', 'UNKNOWN', '...', '15200600010 / 11', '2', 'P01S02', 'FR-N-S081093')
('3', '2020-Jun-26 18:00:25', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L129575_6', '0', 'FB04716251', 'NaN', 'YES', 'UNKNOWN', '...', '15200600010 / 11', '3', 'P01S03', 'FR-N-S081093')
('4', '2020-Jun-26 18:00:25', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L48781_7', '0', 'FB04250855', 'NaN', 'YES', 'UNKNOWN', '...', '15200600010 / 11', '4', 'P01S04', 'FR-N-S081093')
('5', '2020-Jun-26 18:00:25', '1520060001', ' 12', 'RUN', '4ML_10mm_2ND STAGE', 'L99277_7', '0', 'FB04242223', 'NaN', 'YES', 'UNKNOWN', '...', '15200600010 / 12', '5', 'P02S01', 'FR-N-S081093')
('6', '2020-Jun-26 18:00:25'